<a href="https://colab.research.google.com/github/rukmals/crowd_counting_domain_adaption_model_development/blob/main/UCF_QNRF_Data_SetUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from tensorflow.keras.preprocessing import image

import scipy
from scipy.io import loadmat
import glob
import h5py
import time
from sklearn.externals.joblib import Parallel, delayed
from torchvision import datasets, transforms

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
test_setpath = "/content/drive/MyDrive/Model_Test_Data/UCF-QNRF/UCF-QNRF_ECCV18/Train"

In [4]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density
    #print(gt_count)
    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    pts_copy = pts.copy()
    tree = scipy.spatial.KDTree(pts_copy, leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)
    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1] + distances[i][2] + distances[i][3]) * 0.1
        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [5]:
def generate_density_map(img_path):
    img_path = "/content/drive/MyDrive/Model_Test_Data/UCF-QNRF/UCF-QNRF_ECCV18/Train/"+ img_path
    #print(img_path)
    mat_path = img_path.replace('.jpg', '.mat').replace('.','_ann.')
    #print(mat_path)
    mat = scipy.io.loadmat(mat_path)
    imgfile = image.load_img(img_path)
    img = image.img_to_array(imgfile)
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = mat["annPoints"]
    #print(mat)
    #print(gt)
    for i in range(0, len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1
    k = gaussian_filter_density(k)
    output_path = img_path.replace('.jpg', '.h5').replace('Train/','ground-truth-h5-train/')
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    print("output", output_path)
    with h5py.File(output_path, 'w') as hf:
        hf['density'] = k
    return img_path

In [6]:
test_list = os.listdir(test_setpath)
print(test_list)

['img_0027.jpg', 'img_0775_ann.mat', 'img_1082_ann.mat', 'img_0697.jpg', 'img_0600_ann.mat', 'img_0922.jpg', 'img_0691.jpg', 'img_0262_ann.mat', 'img_0097.jpg', 'img_1097_ann.mat', 'img_0155.jpg', 'img_1140_ann.mat', 'img_0521.jpg', 'img_1120.jpg', 'img_0345_ann.mat', 'img_1112_ann.mat', 'img_0174.jpg', 'img_1080.jpg', 'img_0241.jpg', 'img_0249.jpg', 'img_1171.jpg', 'img_0606_ann.mat', 'img_0706.jpg', 'img_0835_ann.mat', 'img_0885.jpg', 'img_0825_ann.mat', 'img_0433_ann.mat', 'img_0558.jpg', 'img_0849.jpg', 'img_0282_ann.mat', 'img_0987.jpg', 'img_0041_ann.mat', 'img_1134_ann.mat', 'img_0963.jpg', 'img_0663.jpg', 'img_0627_ann.mat', 'img_0686.jpg', 'img_0951_ann.mat', 'img_0970_ann.mat', 'img_0484.jpg', 'img_0415.jpg', 'img_0556_ann.mat', 'img_0437.jpg', 'img_0582_ann.mat', 'img_0726_ann.mat', 'img_0871_ann.mat', 'img_0085.jpg', 'img_0867.jpg', 'img_0292_ann.mat', 'img_0804_ann.mat', 'img_0171_ann.mat', 'img_0525.jpg', 'img_0639.jpg', 'img_0204.jpg', 'img_0081_ann.mat', 'img_0184.jpg',

In [7]:
#generate_density_map('img_0001.jpg')

In [8]:
img_test_list = []
for item in test_list:
  if '.jpg' in item:
    img_test_list.append(item)
print(len(img_test_list))

1201


In [9]:
dir = os.listdir("/content/drive/MyDrive/Model_Test_Data/UCF-QNRF/UCF-QNRF_ECCV18/ground-truth-h5-train")
len(dir)

941

In [ ]:
c = 0
for path in img_test_list[940:]:
  generate_density_map(path)
  c = c+1
  print(c)

(2706, 4059)
generate density...
